### Práctica 9
### Tratamiento de datos masivos

Partimos del fichero p9data.csv que tiene los datos obtenidos a partir de los sensores del móvil en el bolsillo de personas que han hecho distintos ejercicios, además de algunos datos personales.

0 -> subir escaleras

1 -> bajarlas

2 -> sentarse

3 -> ponerse de pie

4 -> andar 

5 -> correr

El objetivo de esta práctica es intentar predecir el tipo de movimiento **sin usar la etiqueta (label)**. Para ello se pide elaborar un modelo de clustering basado en kmeans tal y como se especifica a continuación.

Enlaces con documentación:

https://spark.apache.org/docs/2.2.0/ml-clustering.html

https://spark.apache.org/docs/2.2.0/api/python/pyspark.ml.html#pyspark.ml.clustering.KMeans

In [56]:
%matplotlib inline
import os
# cambiamos las variables del sistema
spark = 'C:\\hlocal\\tdm\\spark\\hadoop\\spark-2.3.2-bin-hadoop2.7'
# en el path se añade
path = os.environ.get('PATH') 
path = path+ ';'+spark+'\\bin;'
os.environ['PATH'] = path
os.environ['SPARK_HOME']= spark 
os.environ['HADOOP_HOME']= spark 
os.environ['PYSPARK_DRIVER_PYTHON']= 'jupyter'
os.environ['PYSPARK_DRIVER_PYTHON_OPTS']='notebook'

# si da problema con collect quizás haya que poner java_home
os.environ['JAVA_HOME']= 'C:\\Program Files\\Java\\jdk1.8.0_151'
#labs = 'C:\\JDK\\jdk8-64bits'
#os.environ['JAVA_HOME']= labs
os.environ['PATH'] = os.environ.get('JAVA_HOME')+'\\bin;'+path


import findspark
findspark.init()

import pyspark # only run after findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext
print("Preparado!!")


Preparado!!


**1** Cargar el fichero p9data.csv, y dejarlo en una variable de nombre raw.

In [57]:
#path = 'c:/hlocal/tdm/p9data.csv'
path = 'c:/hlocal/tdm/p9data.csv'

## Solución


In [58]:
# para probarlo
raw.printSchema()

root
 |-- label: integer (nullable = true)
 |-- attitude_roll: integer (nullable = true)
 |-- attitude_pitch: double (nullable = true)
 |-- attitude_yaw: double (nullable = true)
 |-- gravity_x: double (nullable = true)
 |-- gravity_y: double (nullable = true)
 |-- gravity_z: double (nullable = true)
 |-- rotationRate_x: double (nullable = true)
 |-- rotationRate_y: double (nullable = true)
 |-- rotationRate_z: double (nullable = true)
 |-- userAcceleration_x: double (nullable = true)
 |-- userAcceleration_y: double (nullable = true)
 |-- userAcceleration_z: double (nullable = true)
 |-- c: double (nullable = true)



**2** Convertir todas las columnas salvo 'label' en un vector de nombre 'rawfeatures'. Quedarse solo con las columnas label y feaeuteres. El resultado será un dataset con nombre 'datosprep', y que tendrá el esquema

```
datosprep.printSchema()
root
 |-- label: integer (nullable = true)
 |-- rawfeatures: vector (nullable = true)
```

In [2]:
# Solución


# para probar:
# datosprep.printSchema()

**3** Antes de hacer kmeans, hay que escalar los datos. Para esto utilizaremos el transformador MinMaxScaler, que se aplicara como columna de entrada a rawfeatures, y como columna de salida, 'features'. Para hacer esto tendremos que hacer un modelo (al que podemos llamar scalerModel) y aplicarselo al propio dataframe datosprep. El resultado será un dataframe al que podemmos llamar scaledData.

https://spark.apache.org/docs/latest/ml-features.html#minmaxscaler

In [3]:
# Solución


In [61]:
# para probarlo
scaledData.show(5)

+-----+--------------------+--------------------+
|label|         rawfeatures|            features|
+-----+--------------------+--------------------+
|    0|[0.0,1.528132,-0....|[0.0,0.7432113863...|
|    0|[1.0,1.527992,-0....|[6.08939227865059...|
|    0|[2.0,1.527765,-0....|[1.21787845573011...|
|    0|[4.0,1.493941,-0....|[2.43575691146023...|
|    0|[5.0,1.476302,-0....|[3.04469613932529...|
+-----+--------------------+--------------------+
only showing top 5 rows



**4** Crear una variable model para el número de clusters k que se desee

In [62]:
# esto no lo tocamos
k=10

In [63]:
# solución


Tras obtener el modelo vemos el resultado, aplicándoselo al propio conjunto:

In [65]:
predicho = model.transform(scaledData)
predicho.show(10)

+-----+--------------------+--------------------+----------+
|label|         rawfeatures|            features|prediction|
+-----+--------------------+--------------------+----------+
|    0|[0.0,1.528132,-0....|[0.0,0.7432113863...|         7|
|    0|[1.0,1.527992,-0....|[6.08939227865059...|         7|
|    0|[2.0,1.527765,-0....|[1.21787845573011...|         7|
|    0|[4.0,1.493941,-0....|[2.43575691146023...|         7|
|    0|[5.0,1.476302,-0....|[3.04469613932529...|         7|
|    0|[6.0,1.455153,-0....|[3.65363536719035...|         7|
|    0|[8.0,1.44344,-0.6...|[4.87151382292047...|         7|
|    0|[9.0,1.443071,-0....|[5.48045305078553...|         7|
|    0|[10.0,1.434186,-0...|[6.08939227865059...|         7|
|    0|[12.0,1.366432,-0...|[7.30727073438070...|         7|
+-----+--------------------+--------------------+----------+
only showing top 10 rows



**5** [4 puntos] Ahora queremos saber, para cada cluster (entre 0 y k-1), cuantos valores de cada valor para label entre 0 y 5 (ambos inclusive) hay. Dada una etiqueta l y un cluster i, podemos contar el número de elementos etiquetados como l en i mediante
```[python]
predicho.where('label=='+str(l)+' and prediction=='+str(i)).count()
```
El resultado de este paso será un array de arrays, con nombre t, con la forma (para k = 7):
```[Python]
[[0, 0, 38858, 35382, 39264, 3135], 
 [36032, 38998, 35798, 35808, 50736, 24069], 
 [0, 0, 22608, 18246, 0, 0], 
 [0, 0, 41845, 34768, 7406, 0], 
 [47962, 48748, 32880, 32880, 47265, 32677], 
 [294, 3172, 40853, 34688, 50374, 19513], 
 [14631, 27073, 41258, 38068, 51989, 21297]]
```

Nota: Estos son valores de un ejemplo ¡no tienen que salir los mismos! (kmeans tiene un componente aleatorio)

In [66]:
# Solución


[9713, 12231, 197, 8314, 18590, 16094]
[19958, 23058, 8644, 25954, 90720, 22937]
[1992, 1692, 74740, 0, 939, 534]
[387, 2966, 166712, 0, 8941, 2601]
[1536, 1637, 3689, 2407, 2224, 1930]
[17271, 14619, 38, 30435, 28399, 15977]
[9127, 11730, 5, 86575, 28088, 10396]
[19906, 25213, 3, 36297, 33961, 13476]
[12624, 12945, 3, 17194, 20400, 10775]
[6405, 11900, 69, 22664, 14772, 5971]
[[9713, 12231, 197, 8314, 18590, 16094], [19958, 23058, 8644, 25954, 90720, 22937], [1992, 1692, 74740, 0, 939, 534], [387, 2966, 166712, 0, 8941, 2601], [1536, 1637, 3689, 2407, 2224, 1930], [17271, 14619, 38, 30435, 28399, 15977], [9127, 11730, 5, 86575, 28088, 10396], [19906, 25213, 3, 36297, 33961, 13476], [12624, 12945, 3, 17194, 20400, 10775], [6405, 11900, 69, 22664, 14772, 5971]]


**6**  Si tuvieras que elegir con el valor k=10, un clúster para detectar lo mejor posible si alguien está sentándose (label==2)
¿qué número de clúster elegirías? Razona tu respuesta (y no la discutas con tu compañero).

In [ ]:
# Solución